<a href="https://www.kaggle.com/code/xmortian/deep-learning-yolov8?scriptVersionId=260448525" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics opencv-python matplotlib numpy tqdm


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO


In [ ]:
# Load YOLOv8 segmentation model (nano for speed)
model = YOLO("yolov8n-seg.pt")  # or yolov8s-seg.pt for better accuracy


In [ ]:
# Classical PipeLine
def classical_lane_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    mask = np.zeros_like(edges)
    height = image.shape[0]
    polygons = np.array([[(200, height), (1100, height), (550, 250)]])
    cv2.fillPoly(mask, polygons, 255)
    masked = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked, 2, np.pi/180, 100, np.array([]),
                            minLineLength=40, maxLineGap=5)

    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 10)

    combined = cv2.addWeighted(image, 0.8, line_image, 1, 1)
    return combined


In [ ]:
# Path to your TuSimple dataset
dataset_path = "/kaggle/input/tusimple/TUSimple"

# Pick a sample test image
test_img_path = os.path.join(dataset_path, "/kaggle/input/tusimple/TUSimple/test_set/clips/0530/1492630582174670101_0/20.jpg")


In [ ]:
 #STEP 3
tusimple_root = "/kaggle/input/tusimple/TUSimple"
train_clips = os.path.join(tusimple_root, "train_set/clips")
train_labels_1 = os.path.join(tusimple_root, "train_set/label_data_0313.json")
train_labels_2 = os.path.join(tusimple_root, "train_set/label_data_0531.json")
train_labels_3 = os.path.join(tusimple_root, "train_set/label_data_0601.json")


In [ ]:
base_path = "/kaggle/working/tusimple_yolo"
img_train = os.path.join(base_path, "images/train")
img_val = os.path.join(base_path, "images/val")
lbl_train = os.path.join(base_path, "labels/train")
lbl_val = os.path.join(base_path, "labels/val")

for p in [img_train, img_val, lbl_train, lbl_val]:
    os.makedirs(p, exist_ok=True)


In [ ]:
import os
import json
import cv2
import shutil
from tqdm import tqdm

def convert_json_to_yolo(json_path, clips_root, images_out, labels_out):
    with open(json_path, "r") as f:
        for line in tqdm(f, desc=f"Processing {os.path.basename(json_path)}"):
            data = json.loads(line)
            img_rel = data["raw_file"]  # e.g., clips/0531/123/20.jpg
            img_path = os.path.join(clips_root, img_rel.replace("clips/", ""))
            
            # Skip missing images safely
            if not os.path.exists(img_path):
                continue
            
            # Copy image into YOLO dataset
            img_name = os.path.basename(img_path)
            save_img_path = os.path.join(images_out, img_name)
            shutil.copy(img_path, save_img_path)
            
            # Get image size
            image = cv2.imread(img_path)
            height, width = image.shape[:2]
            
            # Create YOLO segmentation label
            label_path = os.path.join(labels_out, img_name.replace(".jpg", ".txt"))
            with open(label_path, "w") as lf:
                for lane in data["lanes"]:
                    points = []
                    for x, y in zip(lane, data["h_samples"]):
                        if x > 0:  # valid point
                            points.append(f"{x/width} {y/height}")
                    if points:
                        lf.write(f"0 {' '.join(points)}\n")


In [ ]:
import os
import json
import cv2
import numpy as np
import shutil
from tqdm import tqdm

# Paths
tusimple_root = "/kaggle/input/tusimple/TUSimple"
clips_root = os.path.join(tusimple_root, "train_set/clips")

# YOLO-formatted dataset folder
base_path = "/kaggle/working/tusimple_yolo"
img_train = os.path.join(base_path, "images/train")
lbl_train = os.path.join(base_path, "labels/train")
os.makedirs(img_train, exist_ok=True)
os.makedirs(lbl_train, exist_ok=True)

# Helper function: Convert TuSimple JSON → YOLO segmentation masks
def create_lane_masks(json_path, clips_root, img_out, lbl_out):
    with open(json_path, "r") as f:
        for line in tqdm(f, desc=f"Processing {os.path.basename(json_path)}"):
            data = json.loads(line)
            img_rel = data["raw_file"]
            img_path = os.path.join(clips_root, img_rel.replace("clips/", ""))

            # Skip missing images safely
            if not os.path.exists(img_path):
                continue

            # Read original image
            image = cv2.imread(img_path)
            height, width = image.shape[:2]

            # Create a blank black mask
            mask = np.zeros((height, width), dtype=np.uint8)

            # Draw lanes on mask
            for lane in data["lanes"]:
                pts = []
                for x, y in zip(lane, data["h_samples"]):
                    if x > 0:  # valid point
                        pts.append((x, y))
                if len(pts) > 1:
                    pts = np.array(pts, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(mask, [pts], isClosed=False, color=255, thickness=8)

            # Save image
            img_name = os.path.basename(img_path)
            save_img_path = os.path.join(img_out, img_name)
            shutil.copy(img_path, save_img_path)

            # Convert mask into YOLO segmentation format
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            label_path = os.path.join(lbl_out, img_name.replace(".jpg", ".txt"))
            with open(label_path, "w") as lf:
                for cnt in contours:
                    if len(cnt) >= 3:  # valid polygon
                        norm_points = []
                        for p in cnt:
                            x, y = p[0]
                            norm_points.append(f"{x/width} {y/height}")
                        lf.write(f"0 " + " ".join(norm_points) + "\n")

# Process all TuSimple training JSONs
train_labels = [
    os.path.join(tusimple_root, "train_set/label_data_0313.json"),
    os.path.join(tusimple_root, "train_set/label_data_0531.json"),
    os.path.join(tusimple_root, "train_set/label_data_0601.json")
]

for json_file in train_labels:
    create_lane_masks(json_file, clips_root, img_train, lbl_train)


In [ ]:
%%writefile /kaggle/working/tusimple.yaml
path: /kaggle/working/tusimple_yolo

train: images/train
val: images/train   # We'll split later if needed

names:
  0: lane


In [ ]:
model = YOLO("yolov8s-seg.pt")
results = model.train(
    data="/kaggle/working/tusimple.yaml",
    epochs=150, # You can keep this high, but patience will stop it early
    imgsz=640,
    batch=8,
)

In [ ]:
import matplotlib.pyplot as plt

# Test image path
test_img = os.path.join(
    tusimple_root,
    "/kaggle/input/tusimple/TUSimple/train_set/clips/0531/1492626287507231547/1.jpg"
)

# Predict lane segmentation
result = model.predict(source=test_img)

# Show YOLO segmentation result
plt.figure(figsize=(10, 6))
plt.imshow(result[0].plot())
plt.axis("off")
plt.title("YOLOv8 Lane Segmentation")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Test image path
test_img = os.path.join(
    tusimple_root,
    "/kaggle/input/tusimple/TUSimple/test_set/clips/0530/1492630582174670101_0/20.jpg"
)

# Predict lane segmentation
result = model.predict(source=test_img)

# Show YOLO segmentation result
plt.figure(figsize=(10, 6))
plt.imshow(result[0].plot())
plt.axis("off")
plt.title("YOLOv8 Lane Segmentation")
plt.show()


In [ ]:
import os
import cv2
from ultralytics import YOLO
from IPython.display import Video

# Paths
tusimple_root = "/kaggle/input/tusimple/TUSimple"
clip_path = os.path.join(tusimple_root, "train_set/clips/0531/1492626287507231547")

# 1️⃣ Create a video from frames
video_path = "/kaggle/working/sample_clip.mp4"
frames = sorted([f for f in os.listdir(clip_path) if f.endswith(".jpg")])

# Read first frame to get video dimensions
first_frame = cv2.imread(os.path.join(clip_path, frames[0]))
height, width, _ = first_frame.shape

# Create video writer using MP4 codec
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_path, fourcc, 30, (width, height))

# Write all frames into video
for frame in frames:
    img = cv2.imread(os.path.join(clip_path, frame))
    out.write(img)
out.release()

print(f"✅ Input video saved: {video_path}")

# 2️⃣ Load trained YOLOv8 segmentation model
model = YOLO("/kaggle/working/runs/segment/train7/weights/best.pt")

# 3️⃣ Run YOLOv8 on the video and force a custom save path
predict_dir = "/kaggle/working/yolo_predictions"
os.makedirs(predict_dir, exist_ok=True)

results = model.predict(
    source=video_path,
    save=True,
    project=predict_dir,
    name="lane_detect",
    imgsz=640
)

# 4️⃣ Locate processed video
predicted_video_path = os.path.join(predict_dir, "lane_detect", "sample_clip.mp4")

if not os.path.exists(predicted_video_path):
    # Sometimes YOLO renames files — find the right one automatically
    for file in os.listdir(os.path.join(predict_dir, "lane_detect")):
        if file.endswith(".mp4"):
            predicted_video_path = os.path.join(predict_dir, "lane_detect", file)
            break

print(f"🎥 Annotated video saved at: {predicted_video_path}")

# 5️⃣ Display processed video
Video(predicted_video_path, embed=True)
